## Run All to load unity tennis with pretrained Model

In [1]:
import os
# os.environ['PATH'] = f"{os.environ['PATH']}:/home/student/.local/bin"
# os.environ['PATH'] = f"{os.environ['PATH']}:/opt/conda/lib/python3.10/site-packages"

os.environ['PATH'] = f"{os.environ['PATH']}:/home/vidy/.local/bin"
os.environ['PATH'] = f"{os.environ['PATH']}:/home/vidy/mambaforge/envs/py310/lib/python3.10/site-packages"


os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [ ]:
!pip install .


Processing /home/vidy/RL_Tennis
  Preparing metadata (setup.py) ... 

In [ ]:
import numpy as np
import torch
import os
from unityagents import UnityEnvironment
from Agent import DDPGAgent

# Parameters
folder_path = 'Report'
GAMMA = 0.99
LEARNING_RATE = 1e-3
TAU = 0.3
DEVICE = 'cuda'
BUFFER_SIZE = 1_000_000
BATCH_SIZE = 256
n_episodes = 10  # For testing, set to 2000 for full evaluation

# Function to load pretrained models
def load_model(agent_1, agent_2, folder_path):
    agent_1.actor_local.load_state_dict(torch.load(os.path.join(folder_path, 'agent1_actor.pth')))
    agent_1.critic_local.load_state_dict(torch.load(os.path.join(folder_path, 'agent1_critic.pth')))
    agent_2.actor_local.load_state_dict(torch.load(os.path.join(folder_path, 'agent2_actor.pth')))
    agent_2.critic_local.load_state_dict(torch.load(os.path.join(folder_path, 'agent2_critic.pth')))

# Initialize Unity environment

env = UnityEnvironment(file_name="Tennis.x86_64")
brain_name = env.brain_names[0]

# Get state and action sizes from the environment
env_info = env.reset(train_mode=True)[brain_name]
state_size = env_info.vector_observations.shape[1]
action_size = 2   


# Instantiate agents
agent_1 = DDPGAgent(state_size, action_size, random_seed=0, device=DEVICE,
                    learning_rate=LEARNING_RATE, tau=TAU, buffer_size=BUFFER_SIZE,
                    BATCH_SIZE=BATCH_SIZE, gamma=GAMMA)
agent_2 = DDPGAgent(state_size, action_size, random_seed=1, device=DEVICE,
                    learning_rate=LEARNING_RATE, tau=TAU, buffer_size=BUFFER_SIZE,
                    BATCH_SIZE=BATCH_SIZE, gamma=GAMMA)

# Load pretrained weights
load_model(agent_1, agent_2, folder_path)

# List to store episode scores
scores_list = []

# Run inference
for i in range(1, n_episodes + 1):
    env_info = env.reset(train_mode=False)[brain_name]  # Visual mode
    states = env_info.vector_observations
    scores = np.zeros(2)  # Scores for two agents

    while True:
        # Each agent acts without noise
        action_1 = agent_1.act(states[0], add_noise=False)
        action_2 = agent_2.act(states[1], add_noise=False)
        actions = np.concatenate((action_1, action_2), axis=0)

        # Step environment
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done

        scores += rewards
        states = next_states

        if np.any(dones):
            break

    max_score = np.max(scores)
    scores_list.append(max_score)
    print(f"Episode {i} - Scores: {scores} - Max: {max_score:.2f}")

# Close environment
env.close()

# Print average score
print(f"Average Score over {n_episodes} episodes: {np.mean(scores_list):.2f}")


Found path: /home/vidy/RL_Tennis/Tennis.x86_64
Mono path[0] = '/home/vidy/RL_Tennis/Tennis_Data/Managed'
Mono config path = '/home/vidy/RL_Tennis/Tennis_Data/MonoBleedingEdge/etc'
Preloaded 'ScreenSelector.so'
Preloaded 'libgrpc_csharp_ext.x64.so'
Unable to preload the following plugins:
	ScreenSelector.so
	libgrpc_csharp_ext.x86.so
Logging to /home/vidy/.config/unity3d/Unity Technologies/Unity Environment/Player.log


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Episode 1 - Scores: [0.1  0.19] - Max: 0.19
Episode 2 - Scores: [0.09 0.1 ] - Max: 0.10
Episode 3 - Scores: [0.2  0.19] - Max: 0.20
Episode 4 - Scores: [2.60000004 2.60000004] - Max: 2.60
Episode 5 - Scores: [2.60000004 2.60000004] - Max: 2.60
Episode 6 - Scores: [-0.01  0.1 ] - Max: 0.10
Episode 7 - Scores: [2.50000004 2.60000004] - Max: 2.60
Episode 8 - Scores: [2.70000004 2.60000004] - Max: 2.70
Episode 9 - Scores: [2.60000004 2.60000004] - Max: 2.60
Episode 10 - Scores: [-0.01  0.1 ] - Max: 0.10
Average Score over 10 episodes: 1.38
